In [1]:
"""
Minimal example: optimize 3D V(x) with tensor train.
See next cell for balanced tree and how to provide starting points.
"""
import qutree
import importlib
import numpy as np
import qutree.optimization

importlib.reload(qutree)
importlib.reload(qutree.optimization)

from qutree.benchmark_functions import *
from qutree import Grid, Objective
from qutree.optimization import (
    MatrixTrainOptimization,
    TensorRankOptimization,
    random_grid_points,
)

In [2]:
# Setup
N, r, f, nsweep = 11, 4, 3, 1
prims = [Grid(np.linspace(0.,1.,N), [k]) for k in range(f)]
g0 = random_grid_points(prims, r)

obj_trc = Objective(double_minimum_exp, lambda x: -np.exp(-x))
obj_mt = Objective(double_minimum_exp, lambda x: -np.exp(-x))

trc = TensorRankOptimization(prims, r)
g_trc, _ = trc.sweep(Grid(g0.grid.copy(), g0.coords.copy()), obj_trc, epoch=0)

mt  = MatrixTrainOptimization(prims, r)
g_mt, _ = mt.sweep(Grid(g0.grid.copy(), g0.coords.copy()), obj_mt, epoch=0)

assert np.allclose(g_trc.grid, g_mt.grid)

vs_trc = np.array([v for v in obj_trc.cache.values()])
f_min_trc = np.min(obj_trc.logger.df['f'].array)

vs_mt = np.array([v for v in obj_mt.cache.values()])
f_min_mt = np.min(obj_mt.logger.df['f'].array)

print(f_min_trc-f_min_mt)

0.0


In [3]:
N, r, f, nsweep = 11, 4, 6, 3

primitive_grid = [Grid(np.linspace(0.0, 1.0, N, endpoint=True), [k]) for k in range(f)]
obj = Objective(double_minimum_exp, lambda x: -np.exp(-x))
model = TensorRankOptimization(primitive_grid, r)

grid = random_grid_points(primitive_grid, r)
grid = model.optimize(grid, obj, nsweep)

vs = np.array([v for v in obj.cache.values()])
f_min = np.min(obj.logger.df["f"].array)

print("Tensor Rank Cross")
print(obj)
print("minimal: ", f_min)
print("____________________________________________")

primitive_grid = [Grid(np.linspace(0.0, 1.0, N, endpoint=True), [k]) for k in range(f)]
obj = Objective(double_minimum_exp, lambda x: -np.exp(-x))
model = MatrixTrainOptimization(primitive_grid, r)

grid = random_grid_points(primitive_grid, r)
grid = model.optimize(grid, obj, nsweep)

vs = np.array([v for v in obj.cache.values()])
f_min = np.min(obj.logger.df["f"].array)

print("Matrix Train Optimization")
print(obj)
print("minimal: ", f_min)

Tensor Rank Cross
Optimal value:
x1       0.800000
x2       0.800000
x3       1.000000
x4       0.600000
x5       0.900000
x6       0.800000
f        0.489823
epoch    0.000000
Name: 19, dtype: float64

Number of objective function calls: 611
Number of cached function accesses: 181
Total number calls: 792
minimal:  0.4898233716608451
____________________________________________
Matrix Train Optimization
Optimal value:
x1       0.600000
x2       0.800000
x3       0.700000
x4       0.900000
x5       0.900000
x6       0.600000
f        0.538066
epoch    0.000000
Name: 71, dtype: float64

Number of objective function calls: 561
Number of cached function accesses: 375
Total number calls: 936
minimal:  0.5380656821206244


In [ ]:
# represent via TRC
# A via 3 options: a.) full matrix, b.) elements only where groups match, c.) diagonal


In [ ]:
"""
Minimal example: optimize 3D V(x) with tensor train.
See next cell for balanced tree and how to provide starting points.
"""
from qutree import *


obj = Objective(double_minimum_nd, lambda x: -np.exp(-0.01*x))
# obj = Objective(double_minimum_exp, lambda x: -np.exp(-0.01*x))
N, r, f, nsweeps, seed = 11, 4, 3, 2, np.random.randint(100000)

# --- Pick a network structure: ---
G = tensor_train_graph(f, r, N)
fig = plot_tt_diagram(G)

# boundaries of primitive grid
primitive_grid = [np.linspace(0., 1., num = N)] * f

G_opt = ttnopt(G, obj, nsweeps, primitive_grid)

# Display output (can vary with random parameters)
print(obj)
display(obj.logger.df)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qutree.benchmark_functions import *

x = np.linspace(-0, 1, 400)
y = np.linspace(-0, 1, 400)
X, Y = np.meshgrid(x, y)
f = rastigrin

Z = np.zeros_like(X)
for i in range(X.shape[0]):
    for j in range(X.shape[1]):
        Z[i, j] = f(np.array([X[i, j], Y[i, j]]))

plt.figure()
plt.contourf(X, Y, Z, levels=50)
plt.colorbar()
plt.show()